In [27]:
import json
import pandas as pd
import ast
import datetime

In [28]:
def display(value):
    s = str(value)
    if len(s) >=4 and len(s)  <= 6:
        return s[0:len(s) - 3] + "K"
    elif len(s) >=7 and len(s)  <= 9:
        return s[0:len(s) - 6] + "M"
    elif len(s) >=10 and len(s)  <= 12:
        return s[0:len(s) - 9] + "B"
    return value 

def convert_unixtime(utime):
    return (datetime.datetime.fromtimestamp(int(utime)).strftime('%d-%m'))

def convert_country_code(code):
    mapping = {"CO":"Colombia", "BR": "Brazil", "VE": "Venezuela", "PE": "Peru", "UY": "Uruguay", 
               "PY": "Paraguay", "PA": "Panama", "CL": "Chile", "EC":"Ecuador", "AR": "Argentina",
               "CR": "Costa Rica", "GY": "Guyana", "MX": "Mexico", "US": "United States",  
              }
    return mapping[code]

def cut(df, col, values, savedcols= ['Location', 'dau_audience', 'mau_audience']):
    df_slice = {}
    
    for i, v in enumerate(values):
        df_slice[i] = df[df[col] == v][savedcols]
    
    merge = pd.merge(df_slice[0], df_slice[1], on=["Location"], suffixes=("_%s" % (values[0]), "_%s" % (values[1])))
    for i in range(2, len(values)):
        #print("Adding suffix for:" + values[i])
        merge = pd.merge(merge, df_slice[i], on=["Location"], suffixes=("", "_%s" % (values[i])))
        
    if len(values) > 2:
        # Fix information for the second group.
        merge["audience_%s" % (values[2])] = merge["audience"]
        del merge["audience"]
    return merge   

def copy_rename(df, oldname, newname):
    return df.rename(columns={oldname: newname}).copy() 

def get_slice(dfin, col, values, frequency="mau"):
    dfout = copy_rename(dfin, '%s_audience' % (frequency), "audience")
    dfout = cut(dfout, col, values, savedcols=['Location', 'audience'])
    dfout["Frequency"] = "Daily" if frequency == "dau" else "Monthly"
    return dfout

def calculate_percentages(df, cols, prefix):
    totals = df.sum(axis=1)
    for col in cols:
        df[prefix + "audience_" + col] = df["audience_" + col] * 100. / totals
        df[prefix + "audience_" + col] = df["audience_" + col] * 100. / totals

In [29]:
#infile = "./collections/immigrants/dataframe_collected_finished_1531383976.csv.gz"  # 12-07
#infile = "./collections/immigrants/dataframe_collected_finished_1531404806.csv.gz"  # 12-07
#infile = "./collections/immigrants/dataframe_collected_finished_1531408020.csv.gz"  # 12-07
infile = "./collections/immigrants/dataframe_collected_finished_1531730873.csv.gz"  # 16-07

df = pd.read_csv(infile)
df["CollectionDay"] = df["timestamp"].apply(lambda x: convert_unixtime(x))
print("Collection made in %s" % (df["CollectionDay"].head()[0]))

Collection made in 16-07


In [30]:
def extract_relationship(d):
    if 1 in d:
        return "single"
    elif 2 in d:
        return "dating"
    elif 3 in d:
        return "married"
    else:
        return None

def extract_education(d):
    if d == [3, 7, 8, 9, 11]:
        return "graduated"
    elif d == [1, 12, 13]:
        return "no_degree"
    elif d == [2, 4, 5, 6, 10]:
        return "high_school"
    elif d == [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
        return "all"
    else:
        return None

def extract_group(d):
    id = d[0]["id"]
    if id == 6026404871583:
        return "Expats (Venezuela)"
    elif id == 6015559470583:
        return "Ex-pats (All)"
    else:
        return "All"

def agebuckets(minage, maxage):
    if minage == 13 and (maxage is None or np.isnan(maxage)):
        return "all"
    elif minage == 13 and maxage == 18:
        return "adolecent"
    elif minage == 19 and maxage == 25:
        return "young_adult"
    elif minage == 26 and maxage == 40:
        return "adult"
    elif minage == 41 and maxage == 65:
        return "middle_age"
    elif minage == 65 and (maxage is None or np.isnan(maxage)):
        return "elder"
    return "undefined"
    
def expand(row):
    place = None
    loc_dimension = None
    if "regions" in row["geo_locations"]:
        place = "%s, %s" % (row["geo_locations"]["regions"][0]["name"], convert_country_code(row["geo_locations"]["regions"][0]["country_code"]))
        loc_dimension = "State"
    elif "countries" in row["geo_locations"]:
        place = convert_country_code(row["geo_locations"]["countries"][0])
        loc_dimension = "Country"
    elif "cities" in row["geo_locations"]:
        place = "%s, %s, %s" % (row["geo_locations"]["cities"][0]["name"], row["geo_locations"]["cities"][0]["region"], convert_country_code(row["geo_locations"]["cities"][0]["country"]))
        loc_dimension = "City"
    
    loctype = "_".join(row["geo_locations"]["location_types"])

    relationship, education, group = None, None, None
    for dimension in row["flexible_spec"]:
        if "relationship_statuses" in dimension:
            relationship = extract_relationship(dimension["relationship_statuses"])
        elif "education_statuses" in dimension:
            education = extract_education(dimension["education_statuses"])
        elif "behaviors" in dimension:
            group = extract_group(dimension["behaviors"])
    
    gender = row["genders"][0]
    gender = "both" if gender == 0 else "man" if gender == 1 else "woman"
    
    return row["age_min"], row["age_max"], place, loc_dimension, loctype, gender, relationship, education, group

def get_item(x):
    if not x:
        return None
    return x["name"]

df[["MinAge","MaxAge","Location","LocationHierarchy","LocationType","Gender","Relationship","Education","Group"]] = df["targeting"].apply(lambda x : expand(ast.literal_eval(x))).apply(pd.Series)

if "citizenship" in df:
    df["Group"] = df["citizenship"].fillna("[]").apply(lambda x : get_item(ast.literal_eval(x)))
    print("Updating Group information...")
    
if "access_device" in df:
    df["Device"] = df["access_device"].fillna("[]").apply(lambda x : get_item(ast.literal_eval(x)))
    print("Adding information regarding devices...")
else:
    df["Device"] = None
    
df["AgeBucket"] = df[["MinAge","MaxAge"]].apply(lambda x: agebuckets(x["MinAge"], x["MaxAge"]), axis=1)

print("Removing redundant cols")
for col in ["Unnamed: 0", "all_fields", "targeting",'behavior', 'citizenship', "mock_response", "access_device", 
            "ages_ranges", "household_composition", 'interests', 'family_statuses', 'genders', 'geo_locations', 'languages',
            'name', 'relationship_statuses', 'response', 'scholarities', 'timestamp', 'publisher_platforms',]:
    if col in df.keys():
        del df[col]

# Gave up using travel_in   
df = df[df["LocationType"] != "travel_in"]

print("All Done!")
# Brief description: 
# -----------------
# Gender: 0 Both, 1 Man, 2 Woman
# Age: 13-Null, 13-18, 19-25, 26-40, 41-65, 65+
# LocationType: home_recent, home, recent


Updating Group information...
Removing redundant cols
All Done!


In [31]:
dfgender = {}
dfcut = df[(df["Gender"].apply(lambda x : x in ["man","woman"])) & (df["Education"].isnull()) & (df["Device"].isnull()) & (df["AgeBucket"] == "all") & (df["Group"] == "Expats (Venezuela)") & (df["LocationType"] == "home_recent")].copy()
dfgender["mau"] = get_slice(dfcut, "Gender", ["man","woman"], frequency="mau")
dfgender["dau"] = get_slice(dfcut, "Gender", ["man","woman"], frequency="dau")
calculate_percentages(dfgender["mau"], ["man","woman"], "%")
calculate_percentages(dfgender["dau"], ["man","woman"], "%")

In [32]:
df_gender_locals = {}
dfcut = df[(df["Gender"].apply(lambda x : x in ["man","woman"])) & (df["Education"].isnull()) & (df["Device"].isnull()) 
           & (df["AgeBucket"] == "all") & (df["Group"].isnull()) & (df["LocationType"] == "home_recent") ].copy()

df_gender_locals["mau"] = get_slice(dfcut, "Gender", ["man","woman"], frequency="mau") 
df_gender_locals["dau"] = get_slice(dfcut, "Gender", ["man","woman"], frequency="dau")

calculate_percentages(df_gender_locals["mau"], ["man","woman"], "%locals_")
calculate_percentages(df_gender_locals["dau"], ["man","woman"], "%locals_")

for col in ["man","woman"]:
    del df_gender_locals["mau"]["audience_" + col]
    del df_gender_locals["dau"]["audience_" + col]


In [33]:
immigrant_groups = ['Expats (Venezuela)', u'Expats (United States)', u'Expats (Mexico)', u'Expats (Brazil)', u'Expats (Argentina)', 
                    u'Expats (Peru)', u'Expats (Chile)', u'Expats (Spain)', u'Expats (France)', u'Expats (United Kingdom)', u'Expats (Colombia)', 
                    u'Expats (Cuba)', u'Expats (Germany)', u'Expats (Italy)', u'Expats (Portugal)', u'Expats (Jamaica)', 'Expats (China)']
df_immi_place = {}
dfcut = df[(df["Group"].apply(lambda x: x in immigrant_groups)) & (df["Gender"] == "both") & (df["Education"].isnull()) & 
           (df["Device"].isnull()) & (df["AgeBucket"] == "all") & (df["LocationType"] == "home_recent")].copy()
df_immi_place["mau"] = get_slice(dfcut, "Group", immigrant_groups, frequency="mau")
df_immi_place["dau"] = get_slice(dfcut, "Group", immigrant_groups, frequency="dau")

calculate_percentages(df_immi_place["mau"], immigrant_groups, "%")
calculate_percentages(df_immi_place["dau"], immigrant_groups, "%")


In [73]:
dffinal = pd.concat([df_immi_place["mau"], df_immi_place["dau"]])

for col in dffinal.keys():
    if col.startswith("%"):
        print(col)
        dffinal[col] = dffinal[col].round(2)

audience_group = []
for group in immigrant_groups:
    audience_group += ["audience_" + group]
dffinal["summary_total_immigrants"] = dffinal[audience_group].sum(axis=1)        

dffinal["display_total_immigrants"] = dffinal["summary_total_immigrants"].apply(lambda x: display(x))
dffinal.to_csv("immigrants_07_17.csv", index=False)

%audience_Expats (Venezuela)
%audience_Expats (United States)
%audience_Expats (Mexico)
%audience_Expats (Brazil)
%audience_Expats (Argentina)
%audience_Expats (Peru)
%audience_Expats (Chile)
%audience_Expats (Spain)
%audience_Expats (France)
%audience_Expats (United Kingdom)
%audience_Expats (Colombia)
%audience_Expats (Cuba)
%audience_Expats (Germany)
%audience_Expats (Italy)
%audience_Expats (Portugal)
%audience_Expats (Jamaica)
%audience_Expats (China)
